# 아이유팬이 좋아할 만한 다른 아티스트 찾기
### 추천 시스템(Recommender System)
나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추천해 준다 
#### 협업 필터링(Collaborative Filtering) 방식
다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악

아이템과 사용자 간의 행동 또는 관계에만 주목할 뿐 아이템 자체의 고유한 속성에 주목하지 않는다.
#### 콘텐츠 기반 필터링(Contents-based Filtering) 방식
아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악

아이템 자체의 속성에만 주목하고 사용자와 아이템 간의 관련성 에는 주목하지 않는다
#### 협업 필터링을 바로 사용할 수 없게 만드는 세가지 제약 조건
- 시스템이 충분한 정보를 모으지 못한 사용자나 아이템에 대한 추론을 할 수 없는 상태인 콜드 스타트(Cold Start) 상황
- 계산량이 너무 많아 추천의 효율이 떨어지는 상황
- 롱테일의 꼬리 부분, 즉 사용자의 관심이 저조한 항목의 정보가 부족하여 추천에서 배제되는 상황

#### 아이템은 매우 많고 유저의 취향은 다양할 떄 유저가 소비할 만한 아이템을 예측하는 모델이 추천 시스템이다

## 데이터 탐색하기와 전처리
데이터에는 컬럼명이 포함되어 있지 않으므로, 아래와 같이 col_names에 적절히 컬럼명을 지정

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


사용하는 컬럼만 남기고 제거

In [2]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [3]:
#검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔
data['artist'] = data['artist'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
#첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


#### 데이터 탐색
- 유저 수, 아티스트 수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

(참고) pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용

In [5]:
# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

#### 모델 검증을 위한 사용자 초기 정보 세팅
우리가 만든 추천 시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 함.

좋아하는 유명한 아티스트들을 기존의 데이터에 추가

In [10]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


#### 모델에 활용하기 위한 전처리
user와 artist 각각에 번호를 붙이는 indexing 작업을 합니다.
 
추천 시스템, 자연어 처리에서 자주 하는 작업

(참고) pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용

In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [12]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


## 사용자의 명시적/암묵적 평가
#### 암묵적 피드백 데이터셋의 특징
- 부정적인 피드백이 없다
- 애초에 잡음이 많다.
- 수치는 신뢰도를 의미한다.
- Implicit-feedback Recommender System의 평가는 적절한 방법을 고민해봐야 한다.

In [14]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}') 
# f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


## Matrix Factorization(MF, 행렬 분해)
(m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 아이디어

k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리

아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델

## CSR(Compressed Sparse Row) Matrix
사용하고 계실 컴퓨터의 메모리는 많아야 16GB일 테니 97GB나 되는 거대한 행렬을 메모리에 올려놓고 작업한다는 것은 불가능, 이런 경우의 좋은 대안이 되는 것이 CSR(Compressed Sparse Row) Matrix

CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조

CSR Matrix는 data, indices, indptr 로 행렬을 압축하여 표현

In [15]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

## MF 모델 학습하기
implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지

이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용

Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있다.

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

- factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 함.

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

- Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [20]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [21]:
zimin_vector

array([ 0.8039702 , -1.3538158 ,  0.19322494, -0.61515456, -0.0255734 ,
        0.668688  ,  0.221062  ,  0.32768154,  1.0826072 ,  0.81981987,
        0.40105247,  1.7780246 ,  0.2702248 , -0.6796795 , -0.8302564 ,
        0.26323938,  0.28362858,  0.50267005,  0.448167  ,  0.4145496 ,
        0.29052427,  0.54287976,  0.761158  , -0.09169633,  0.57646185,
       -0.9843685 , -0.39445013,  0.6461321 ,  0.27157328, -0.5500492 ,
        0.05621582, -0.837981  ,  0.820517  ,  0.41748878,  0.02398882,
       -0.7450416 , -1.188102  , -0.402093  , -1.1123505 , -0.04183268,
       -0.50119483,  0.08036307,  0.01594627,  0.44959065,  0.30121264,
       -0.19647917, -0.26043898, -1.259148  ,  0.07523334, -0.34247583,
        0.19121544, -0.1492447 , -1.2825111 ,  0.08665143, -0.6579383 ,
       -0.36609855, -0.15403028,  0.67368275,  0.26014423, -0.27105722,
       -0.8237621 ,  0.33586353,  1.3177516 ,  0.4780406 , -0.7886104 ,
       -0.00338234, -0.07061656, -0.7626253 , -0.47929054,  0.75

In [22]:
black_eyed_peas_vector

array([ 1.01122595e-02, -4.37333575e-03,  1.09775634e-02, -2.25713383e-03,
        8.07218254e-03,  1.17588192e-02,  7.53091695e-03,  1.18988450e-03,
        8.22809711e-03,  2.76304409e-02, -1.53509097e-03,  2.20161080e-02,
       -3.57818702e-04,  1.29544390e-02, -1.42856771e-02,  8.11502524e-03,
        1.73303578e-02,  1.20907184e-02,  1.86371282e-02,  1.63440742e-02,
       -4.72748885e-03,  7.17133656e-03,  5.60887624e-03, -4.19838587e-04,
        1.91026535e-02, -1.16191702e-02, -9.67067387e-03,  1.36311660e-02,
        8.61182902e-03, -4.13226569e-03,  1.29778432e-02,  1.43233072e-02,
        7.16371462e-03,  8.93105287e-03,  1.81249622e-02, -9.75913741e-03,
       -1.14846546e-02,  2.72983173e-03, -1.54045620e-03,  7.29212537e-03,
       -1.22277075e-02,  1.72362309e-02,  8.82168487e-03,  5.71777309e-05,
        2.41194870e-02,  4.54022130e-03,  5.15450584e-03, -6.30975375e-03,
        1.74291022e-02, -5.67326602e-03,  1.43675115e-02,  1.26393409e-02,
       -8.58919322e-03,  

In [23]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.49118546

모델이 저의 queen에 대한 선호도를 어떻게 예측할지 확인

In [24]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.3113567

## 비슷한 아티스트 찾기 + 유저에게 추천하기
#### 비슷한 아티스트 찾기
AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾기

In [25]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 1.0),
 (277, 0.9868435),
 (28, 0.9790089),
 (5, 0.9788992),
 (217, 0.9742237),
 (247, 0.9713931),
 (490, 0.9666869),
 (473, 0.96422416),
 (418, 0.95984715),
 (268, 0.9562464),
 (910, 0.9547438),
 (1018, 0.94831795),
 (75, 0.943823),
 (694, 0.94145375),
 (773, 0.939765)]

(아티스트의 id, 유사도) Tuple 로 반환

아티스트의 id를 다시 아티스트의 이름으로 매핑

In [26]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'the beatles',
 'oasis',
 'placebo',
 'u2',
 'pink floyd',
 'nirvana',
 'the smashing pumpkins',
 'queen',
 'foo fighters',
 'nine inch nails']

In [27]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [28]:
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'dr. dre',
 'nas',
 'the game',
 'snoop dogg',
 'ice cube',
 'ludacris',
 'jay-z',
 'busta rhymes']

#### 마니아는 데이터상에서 표현될 때 특징
- 마니아들은 특정 장르의 아티스트들에게로 선호도가 집중
- 다른 장르의 아티스트들과는 선호도가 낮게 나타날 것
- 이런 마니아들의 존재로 인해 같은 장르의 아티스트들의 벡터들도 더 가까워져서 get_similar_artist시 장르별 특성이 두드러지게 될 것

In [29]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'kelly clarkson',
 'leona lewis',
 'nelly furtado']

#### 유저에게 아티스트 추천
AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천

filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument

In [30]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.44949588),
 (550, 0.43093443),
 (369, 0.4267748),
 (354, 0.41305247),
 (1800, 0.41050157),
 (627, 0.40997088),
 (2249, 0.4033073),
 (901, 0.40053037),
 (409, 0.39946765),
 (391, 0.39158115),
 (355, 0.3764247),
 (5555, 0.36899674),
 (274, 0.36312228),
 (944, 0.36169854),
 (382, 0.35913324),
 (618, 0.3585473),
 (724, 0.35347018),
 (564, 0.35114634),
 (28, 0.34525383),
 (24, 0.34503633)]

In [31]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'britney spears',
 'justin timberlake',
 'nelly furtado',
 'lady gaga',
 'maroon 5',
 'katy perry',
 'pink',
 'amy winehouse',
 'christina aguilera',
 'madonna',
 'timbaland',
 'michael jackson',
 'avril lavigne',
 'mika',
 'the pussycat dolls',
 'lily allen',
 'kanye west',
 'the killers',
 'jack johnson']

AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다

In [32]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도를 반환

In [33]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.23149388392047363),
 ('black eyed peas', 0.12630972288425962),
 ('coldplay', 0.04838412568590657),
 ('jason mraz', 0.044925042037494096),
 ('maroon5', 0.00015387336538978736)]

# 프로젝트 - Movielens 영화 추천 실습
이번 프로젝트에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있다
- 별점을 시청횟수로 해석해서 생각
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

### 데이터 준비와 전처리

In [34]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np


In [35]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [36]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [37]:
del ratings['timestamp']

In [38]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [39]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [40]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [41]:
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [42]:
del movies['genre']

In [43]:
movies.head()

,movie_id,title
0,1,toy story (1995)
1,2,jumanji (1995)
2,3,grumpier old men (1995)
3,4,waiting to exhale (1995)
4,5,father of the bride part ii (1995)


In [44]:
title = movies['title'].values
title

array(['toy story (1995)', 'jumanji (1995)', 'grumpier old men (1995)',
       ..., 'tigerland (2000)', 'two family house (2000)',
       'contender, the (2000)'], dtype=object)

In [45]:
movie_id = movies['movie_id'].values
movie_id

array([   1,    2,    3, ..., 3950, 3951, 3952])

In [46]:
movie_dic = { movie_id:title for movie_id, title in zip(movie_id, title) }
movie_dic

{1: 'toy story (1995)',
 2: 'jumanji (1995)',
 3: 'grumpier old men (1995)',
 4: 'waiting to exhale (1995)',
 5: 'father of the bride part ii (1995)',
 6: 'heat (1995)',
 7: 'sabrina (1995)',
 8: 'tom and huck (1995)',
 9: 'sudden death (1995)',
 10: 'goldeneye (1995)',
 11: 'american president, the (1995)',
 12: 'dracula: dead and loving it (1995)',
 13: 'balto (1995)',
 14: 'nixon (1995)',
 15: 'cutthroat island (1995)',
 16: 'casino (1995)',
 17: 'sense and sensibility (1995)',
 18: 'four rooms (1995)',
 19: 'ace ventura: when nature calls (1995)',
 20: 'money train (1995)',
 21: 'get shorty (1995)',
 22: 'copycat (1995)',
 23: 'assassins (1995)',
 24: 'powder (1995)',
 25: 'leaving las vegas (1995)',
 26: 'othello (1995)',
 27: 'now and then (1995)',
 28: 'persuasion (1995)',
 29: 'city of lost children, the (1995)',
 30: 'shanghai triad (yao a yao yao dao waipo qiao) (1995)',
 31: 'dangerous minds (1995)',
 32: 'twelve monkeys (1995)',
 33: 'wings of courage (1995)',
 34: 'babe (1

In [47]:
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [48]:
ratings_movie_id = ratings['movie_id'].values
ratings_movie_id

array([1193,  661,  914, ...,  562, 1096, 1097])

In [49]:
2 in movie_dic.keys()

True

In [50]:
movie_dic[2]

'jumanji (1995)'

In [51]:
ratings_movie_name = []
for i in ratings_movie_id:
    if i in movie_dic.keys():
        ratings_movie_name.append(movie_dic[i])

In [52]:
ratings_movie_name

["one flew over the cuckoo's nest (1975)",
 'james and the giant peach (1996)',
 'my fair lady (1964)',
 'erin brockovich (2000)',
 "bug's life, a (1998)",
 'princess bride, the (1987)',
 'ben-hur (1959)',
 'christmas story, a (1983)',
 'snow white and the seven dwarfs (1937)',
 'wizard of oz, the (1939)',
 'beauty and the beast (1991)',
 'gigi (1958)',
 'miracle on 34th street (1947)',
 "ferris bueller's day off (1986)",
 'sound of music, the (1965)',
 'airplane! (1980)',
 'tarzan (1999)',
 'bambi (1942)',
 'awakenings (1990)',
 'big (1988)',
 'pleasantville (1998)',
 'wallace & gromit: the best of aardman animation (1996)',
 'back to the future (1985)',
 "schindler's list (1993)",
 'meet joe black (1998)',
 'pocahontas (1995)',
 'e.t. the extra-terrestrial (1982)',
 'titanic (1997)',
 'ponette (1996)',
 'close shave, a (1995)',
 'antz (1998)',
 'girl, interrupted (1999)',
 'hercules (1997)',
 'aladdin (1992)',
 'mulan (1998)',
 'hunchback of notre dame, the (1996)',
 'last days of di

In [53]:
ratings['name'] = ratings_movie_name

In [54]:
ratings.head()

,user_id,movie_id,counts,name
0,1,1193,5,one flew over the cuckoo's nest (1975)
1,1,661,3,james and the giant peach (1996)
2,1,914,3,my fair lady (1964)
3,1,3408,4,erin brockovich (2000)
4,1,2355,5,"bug's life, a (1998)"


In [55]:
del ratings['movie_id']

In [56]:
ratings.head()

,user_id,counts,name
0,1,5,one flew over the cuckoo's nest (1975)
1,1,3,james and the giant peach (1996)
2,1,3,my fair lady (1964)
3,1,4,erin brockovich (2000)
4,1,5,"bug's life, a (1998)"


### 분석해 봅시다.
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [57]:
print(ratings['name'].nunique())

3628


In [58]:
print(movies['movie_id'].nunique())  # ratings에 있는 유니크한 영화 개수

3883


In [59]:
print(ratings['user_id'].nunique())  # rating에 있는 유니크한 사용자 수

6039


In [60]:
# 가장 인기 있는 영화 30개

movies_count = ratings.groupby('name')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

name
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pri

### 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다

In [61]:
my_favorite = ['jumanji (1995)', 'superman (1978)', 'Walking Dead, The (1995)', 'Lion King, The (1994)', 'Terminator, The (1984)']

In [62]:
my_playlist = pd.DataFrame({'user_id': ['6041']*5, 'name': my_favorite, 'counts': [5]*5})

if not ratings.isin({'user_id': ['6041']})['user_id'].any():
    ratings = ratings.append(my_playlist)
    
ratings.tail(10)

,user_id,counts,name
1000203,6040,3,platoon (1986)
1000205,6040,5,"crying game, the (1992)"
1000206,6040,5,welcome to the dollhouse (1995)
1000207,6040,4,sophie's choice (1982)
1000208,6040,4,e.t. the extra-terrestrial (1982)
0,6041,5,jumanji (1995)
1,6041,5,superman (1978)
2,6041,5,"Walking Dead, The (1995)"
3,6041,5,"Lion King, The (1994)"
4,6041,5,"Terminator, The (1984)"


### CSR matrix를 직접 만들어 봅시다.

In [66]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['name'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자이다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [67]:
print(user_to_idx['6041'])

6039


In [69]:
print(movie_to_idx['jumanji (1995)'])

513


In [70]:
temp_user_ratings = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_ratings) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_ratings   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_ratings = ratings['name'].map(movie_to_idx.get).dropna()
if len(temp_movie_ratings) == len(ratings):
    print('movie column indexing OK!!')
    ratings['name'] = temp_movie_ratings
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,counts,name
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,5,513
1,6039,5,526
2,6039,5,3628
3,6039,5,3629


In [71]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['name'].nunique()

csr_ratings = csr_matrix((ratings.counts, (ratings.user_id, ratings.name)), shape=(num_user, num_movie))
csr_ratings

<6040x3631 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [72]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=1000, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_ratings.T
csr_data_transpose

<3631x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [73]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

### 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [74]:
my, jumanji = user_to_idx['6041'], movie_to_idx['jumanji (1995)']
my_vector, jumanji_vector = als_model.user_factors[my], als_model.item_factors[jumanji]

print('슝=3')

슝=3


In [75]:
my_vector
my_vector.shape

(1000,)

In [76]:
jumanji_vector
jumanji_vector.shape

(1000,)

In [77]:
np.dot(my_vector, jumanji_vector)

0.93060285

In [78]:
superman_vector = als_model.item_factors[movie_to_idx['superman (1978)']]
np.dot(my_vector, superman_vector)

0.9783531

## 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [83]:
favorite_movie = 'jumanji (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(513, 1.0),
 (3629, 0.4003732),
 (3628, 0.40025306),
 (3630, 0.39956903),
 (3626, 0.38085127),
 (3592, 0.3776962),
 (3559, 0.37650275),
 (3617, 0.37554482),
 (3373, 0.37511608),
 (3536, 0.37340134),
 (3438, 0.37331635),
 (3472, 0.37307164),
 (3581, 0.37293863),
 (3518, 0.37293682),
 (3479, 0.37270653)]

In [84]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['jumanji (1995)',
 'Lion King, The (1994)',
 'Walking Dead, The (1995)',
 'Terminator, The (1984)',
 'five wives, three secretaries and me (1998)',
 'dangerous game (1993)',
 'zachariah (1971)',
 'slappy and the stinkers (1998)',
 'railroaded! (1947)',
 'last klezmer: leopold kozlowski, his life and music, the (1995)',
 'bonheur, le (1965)',
 'show, the (1995)',
 'death in the garden (mort en ce jardin, la) (1956)',
 'napoleon and samantha (1972)',
 'yankee zulu (1994)']

In [86]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [87]:
get_similar_movie('jumanji (1995)')

['jumanji (1995)',
 'Lion King, The (1994)',
 'Walking Dead, The (1995)',
 'Terminator, The (1984)',
 'five wives, three secretaries and me (1998)',
 'dangerous game (1993)',
 'zachariah (1971)',
 'slappy and the stinkers (1998)',
 'railroaded! (1947)',
 'last klezmer: leopold kozlowski, his life and music, the (1995)']

## 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [88]:
user = user_to_idx['6041']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=True)
movie_recommended

[(1130, 0.0852669),
 (763, 0.07443544),
 (714, 0.07352677),
 (596, 0.059247732),
 (2520, 0.058142103),
 (610, 0.055721287),
 (1698, 0.055056192),
 (1238, 0.053806882),
 (1832, 0.05333592),
 (2216, 0.050885875),
 (1267, 0.04955744),
 (328, 0.049378596),
 (1463, 0.049344834),
 (2336, 0.048112806),
 (992, 0.047383435),
 (1829, 0.046763543),
 (410, 0.046703354),
 (518, 0.046692923),
 (2340, 0.04640259),
 (451, 0.045970224)]

In [89]:
[idx_to_movie[i[0]] for i in movie_recommended]

['indian in the cupboard, the (1995)',
 'superman ii (1980)',
 'waterworld (1995)',
 'hook (1991)',
 'it came from hollywood (1982)',
 'trekkies (1997)',
 'rambo: first blood part ii (1985)',
 'mighty joe young (1998)',
 'grand illusion (grande illusion, la) (1937)',
 'max dugan returns (1983)',
 'threesome (1994)',
 "pete's dragon (1977)",
 'bananas (1971)',
 'stalingrad (1993)',
 'live and let die (1973)',
 'happy, texas (1999)',
 "william shakespeare's romeo and juliet (1996)",
 'doctor dolittle (1967)',
 'one magic christmas (1985)',
 'american history x (1998)']

In [96]:
sm = movie_to_idx['indian in the cupboard, the (1995)']
explain = als_model.explain(user, csr_ratings, itemid=kz)

sm

1130

In [97]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('jumanji (1995)', 0.08329502584683898),
 ('superman (1978)', 0.01388025902432546),
 ('Lion King, The (1994)', -0.004041199562756514),
 ('Walking Dead, The (1995)', -0.004055344393079078),
 ('Terminator, The (1984)', -0.0041094456691093545)]

# 회고
## 힘들었던 점
데이터 전처리 부분이 어렵게 느껴졌다. 그리고 implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지라고 하는데 굉장히 학습이 빨리 되길래 놀랐다.


#### 추천 시스템(Recommender System)
나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추천해 준다 
#### 협업 필터링(Collaborative Filtering) 방식
다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악

아이템과 사용자 간의 행동 또는 관계에만 주목할 뿐 아이템 자체의 고유한 속성에 주목하지 않는다.
#### 콘텐츠 기반 필터링(Contents-based Filtering) 방식
아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악

아이템 자체의 속성에만 주목하고 사용자와 아이템 간의 관련성 에는 주목하지 않는다
#### 협업 필터링을 바로 사용할 수 없게 만드는 세가지 제약 조건
- 시스템이 충분한 정보를 모으지 못한 사용자나 아이템에 대한 추론을 할 수 없는 상태인 콜드 스타트(Cold Start) 상황
- 계산량이 너무 많아 추천의 효율이 떨어지는 상황
- 롱테일의 꼬리 부분, 즉 사용자의 관심이 저조한 항목의 정보가 부족하여 추천에서 배제되는 상황

#### 아이템은 매우 많고 유저의 취향은 다양할 떄 유저가 소비할 만한 아이템을 예측하는 모델이 추천 시스템이다

#### Matrix Factorization(MF, 행렬 분해)
(m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 아이디어

k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리

아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델
#### CSR(Compressed Sparse Row) Matrix
사용하고 계실 컴퓨터의 메모리는 많아야 16GB일 테니 97GB나 되는 거대한 행렬을 메모리에 올려놓고 작업한다는 것은 불가능, 이런 경우의 좋은 대안이 되는 것이 CSR(Compressed Sparse Row) Matrix

CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조

CSR Matrix는 data, indices, indptr 로 행렬을 압축하여 표현